# Exploratory Data Analysis and Data Preparation

This notebook presents a comprehensive exploratory data analysis (EDA) and data preparation pipeline for the red wine quality classification task.

## 1. Dataset Overview and Initial Analysis

We begin our analysis with a dataset containing 1,599 wine samples and 11 physicochemical features, along with a target variable representing wine quality. The features include fixed acidity, volatile acidity, citric acid, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, density, pH, sulphates, and alcohol. We performed an initial analysis and found that there were no missing values in the dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

In [ ]:
# Load the dataset
# The CSV header has a special format with quotes, so we need to parse it carefully
with open('data/raw/winequality-red.csv', 'r') as f:
    header_line = f.readline().strip()

# Extract column names from the header
header_clean = header_line.replace('""', '"').replace('"', '')
columns = [col.strip() for col in header_clean.split(';')]

# Read the data
df = pd.read_csv('data/raw/winequality-red.csv', sep=';', skiprows=1, names=columns)

print(f"Dataset shape: {df.shape}")
print(f"Total samples: {len(df)}")
print(f"Total features: {len(df.columns) - 1}")
print(f"\nFeature names:")
for i, col in enumerate(df.columns[:-1], 1):
    print(f"{i:2d}. {col}")

In [ ]:
# Check for missing values
missing = df.isnull().sum()
print("Missing values per column:")
print(missing[missing > 0] if missing.sum() > 0 else "No missing values found!")

The distribution of feature values is summarized in the following table. An additional visualization can be found in the appendix (feature distributions) that allows us to understand the distribution of each feature, including symmetry, potential outliers, and other characteristics, which complements the table and can be verified by the reader.

In [ ]:
# Statistical summary of all features
statistical_summary = df.describe()

# Create a more beautiful table (stargazer style)
fig, ax = plt.subplots(figsize=(14, 6))
ax.axis('off')

# Prepare data
table_data = statistical_summary.round(2).values
row_labels = ['Count', 'Mean', 'Std Dev', 'Min', '25%', 'Median', '75%', 'Max']
col_labels = statistical_summary.columns.tolist()

# Create table with better styling
table = ax.table(cellText=table_data,
                 rowLabels=row_labels,
                 colLabels=col_labels,
                 cellLoc='center',
                 loc='center',
                 bbox=[0, 0, 1, 1])

# Style the table
table.auto_set_font_size(False)
table.set_fontsize(8)

# Header row styling
for i in range(len(col_labels)):
    cell = table[(0, i)]
    cell.set_facecolor('#4472C4')
    cell.set_text_props(weight='bold', color='white')
    cell.set_height(0.08)

# Row labels styling
for i in range(len(row_labels)):
    cell = table[(i+1, -1)]
    cell.set_facecolor('#E7E6E6')
    cell.set_text_props(weight='bold')
    cell.set_width(0.12)

# Alternate row colors for better readability
for i in range(len(row_labels)):
    for j in range(len(col_labels)):
        cell = table[(i+1, j)]
        if i % 2 == 0:
            cell.set_facecolor('#F2F2F2')
        else:
            cell.set_facecolor('white')
        cell.set_height(0.06)

ax.set_title('Statistical Summary of Features', fontsize=14, fontweight='bold', pad=15)

plt.tight_layout()
plt.savefig('figures/statistical_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("Statistical Summary of Features")
print("=" * 80)
statistical_summary

## 2. Target Variable Analysis and Feature Correlations

Wine quality serves as the target variable, taking discrete values from 3 to 8. Understanding the distribution of wine quality is essential for addressing the challenges inherent in classification tasks, particularly given the class imbalance observed in the dataset. Additionally, analyzing feature relationships is critical for identifying multicollinearity and understanding the underlying data structure that may influence model performance.

**Note:** As a team, we read the codebook and documentation of the data (Cortez et al., 2009; UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/wine+quality) and although in the description of the dictionaries the source states that quality can take values from 0 to 10, in our dataset we only found possible values from 3 to 8.

In [ ]:
# Target variable distribution
quality_dist = df['quality'].value_counts().sort_index()
total = len(df)

# Create frequency table
frequency_table = pd.DataFrame({
    'Quality': quality_dist.index,
    'Count': quality_dist.values,
    'Percentage': (quality_dist.values / total * 100).round(2)
})

print("TARGET VARIABLE (QUALITY) DISTRIBUTION")
print("=" * 60)
print(frequency_table.to_string(index=False))
print(f"\nQuality range: {df['quality'].min()} to {df['quality'].max()}")
print(f"Quality values found: {sorted(df['quality'].unique())}")

In [ ]:
# Create combined figure: quality bar chart (left) and correlation matrix (right)
# Significantly increase size for PDF readability
fig, axes = plt.subplots(1, 2, figsize=(24, 10), gridspec_kw={'width_ratios': [0.6, 1.6], 'wspace': 0.6})

# Left: Quality distribution bar chart
ax1 = axes[0]
bars = ax1.bar(quality_dist.index, quality_dist.values, 
               color='crimson', alpha=0.7, edgecolor='black', linewidth=1.5)
ax1.set_xlabel('Quality Score', fontsize=13, fontweight='bold')
ax1.set_ylabel('Count', fontsize=13, fontweight='bold')
ax1.set_title('Quality Score Distribution', fontsize=15, fontweight='bold', pad=12)
ax1.grid(axis='y', alpha=0.3, linestyle='--')
ax1.set_xticks(quality_dist.index)
ax1.tick_params(axis='both', labelsize=11)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}\n({height/total*100:.1f}%)',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# Right: Correlation matrix - make it much bigger for PDF
correlation_matrix = df.corr()
ax2 = axes[1]
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
heatmap = sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=0.25, 
            cbar_kws={"shrink": 0.8, "aspect": 8, "label": ""}, 
            annot_kws={'size': 9, 'weight': 'bold'}, ax=ax2)
ax2.set_title('Correlation Matrix', fontsize=15, fontweight='bold', pad=15)
ax2.set_xticks(range(len(correlation_matrix.columns)))
ax2.set_xticklabels(correlation_matrix.columns, fontsize=10, rotation=90)
ax2.set_yticks(range(len(correlation_matrix.columns)))
ax2.set_yticklabels(correlation_matrix.columns, fontsize=10, rotation=0)

# Adjust colorbar font size
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=10)

# Use subplots_adjust for better control
plt.subplots_adjust(left=0.04, right=0.98, top=0.85, bottom=0.08, wspace=0.6)
plt.savefig('figures/quality_and_correlation_combined.png', dpi=300, bbox_inches='tight')
plt.show()

The bar chart (left) reveals that this variable exhibits class imbalance, with quality levels 5 and 6 being the most frequent, respectively, while there are very few wines with low quality (such as 3) or extremely high quality (such as 8). The order of magnitude of the differences is very significant, as in our dataset there is a probability of 0.63% of having a wine with quality 3, while the probability is more than 68 times greater for quality 5 (42.59%). Similarly, quality 6 has a probability of 39.90%, which is more than 63 times greater than quality 3.

The correlation matrix (right) provides a comprehensive view of pairwise linear relationships between all physicochemical features, enabling us to assess both feature-feature interactions and feature-target associations simultaneously. Notable pairwise correlations between features include strong positive correlations between fixed acidity and citric acid (0.67), fixed acidity and density (0.67), and total sulfur dioxide and free sulfur dioxide (0.67). Strong negative correlations are observed between fixed acidity and pH (-0.68), citric acid and volatile acidity (-0.55), and citric acid and pH (-0.54). These relationships indicate potential multicollinearity that may influence model performance. By examining the quality row in the correlation matrix, we identify the strongest associations with wine quality. Alcohol content shows the strongest positive correlation (0.48), followed by sulphates (0.25) and citric acid (0.23). Conversely, volatile acidity exhibits the strongest negative correlation (-0.39), indicating that higher levels are associated with lower quality scores. Other features showing moderate negative correlations include total sulfur dioxide (-0.19) and density (-0.17). While quality is an ordinal variable, we compute Pearson correlation coefficients given its natural ordering, which allows us to capture linear trends. These correlations provide an initial indication of feature importance, though the classification models will ultimately determine the true discriminative power of each feature.

In [ ]:
# Calculate class imbalance ratios
print("\nClass Imbalance Analysis:")
print("=" * 60)
for q in sorted(quality_dist.index):
    count = quality_dist[q]
    prob = count / total
    print(f"Quality {q}: {count:4d} samples, P = {prob:.4f} ({prob*100:.2f}%)")

print(f"\nRatio Quality 5 to Quality 3: {quality_dist[5] / quality_dist[3]:.2f}x")
print(f"Ratio Quality 6 to Quality 3: {quality_dist[6] / quality_dist[3]:.2f}x")
print(f"Ratio Quality 5 to Quality 8: {quality_dist[5] / quality_dist[8]:.2f}x")
print(f"Ratio Quality 6 to Quality 8: {quality_dist[6] / quality_dist[8]:.2f}x")

## Appendix: Additional Visualizations

### A1. Statistical Summary Table

The following table provides comprehensive descriptive statistics for all features in the dataset, including count, mean, standard deviation, minimum, maximum, and quartiles.

![Statistical Summary of Features](figures/statistical_summary.png)

*Table A1: Statistical Summary of Features*

### A2. Feature Distributions

The following visualization provides a comprehensive view of the distribution of each feature, including symmetry, potential outliers, and other characteristics that complement the statistical summary table.

In [ ]:
# Distribution of all features
features = df.columns[:-1]  # All columns except 'quality'
n_features = len(features)
n_cols = 4
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 2.5*n_rows))
axes = axes.flatten()

for i, feature in enumerate(features):
    axes[i].hist(df[feature], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
    axes[i].set_title(f'{feature}', fontsize=10, fontweight='bold')
    axes[i].set_xlabel('Value', fontsize=8)
    axes[i].set_ylabel('Frequency', fontsize=8)
    axes[i].grid(axis='y', alpha=0.3)
    axes[i].tick_params(labelsize=7)

# Hide extra subplots
for i in range(n_features, len(axes)):
    axes[i].axis('off')

plt.suptitle('Feature Distributions', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99], hpad=0.3, wpad=0.3)
plt.savefig('figures/feature_distributions.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close()

![Feature Distributions](figures/feature_distributions.png)

*Figure A2: Distribution of all features showing histograms for each physicochemical property*

### Feature Correlation Analysis

Understanding the relationships between features is crucial for comprehending the underlying structure of the dataset and identifying potential multicollinearity issues that may affect model performance. The correlation matrix provides a comprehensive view of pairwise linear relationships between all physicochemical features, enabling us to assess both feature-feature interactions and feature-target associations simultaneously.

In [ ]:
# Correlation matrix
correlation_matrix = df.corr()

plt.figure(figsize=(3.5, 3))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
heatmap = sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=0.15, cbar_kws={"shrink": 0.35, "aspect": 25, "label": ""}, annot_kws={'size': 3.5})
plt.title('Correlation Matrix', fontsize=8, fontweight='bold', pad=4)
plt.xticks(fontsize=4.5, rotation=90)
plt.yticks(fontsize=4.5, rotation=0)

# Adjust colorbar font size
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=4)

plt.tight_layout()
plt.savefig('figures/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

By examining the quality row (or column) in the correlation matrix, we identify the strongest associations with wine quality. Alcohol content shows the strongest positive correlation (0.48), followed by sulphates (0.25) and citric acid (0.23). Conversely, volatile acidity exhibits the strongest negative correlation (-0.39), indicating that higher levels are associated with lower quality scores. Other features showing moderate negative correlations include total sulfur dioxide (-0.19) and density (-0.17). While quality is an ordinal variable, we compute Pearson correlation coefficients given its natural ordering, which allows us to capture linear trends. These correlations provide an initial indication of feature importance, though the classification models will ultimately determine the true discriminative power of each feature.

### A4. PCA Variance Analysis

The scree plot and cumulative explained variance plot provide detailed information about the variance explained by each principal component and the number of components needed to retain a specified percentage of the total variance.

![PCA Variance Analysis](figures/pca_variance_analysis.png)

*Figure A4: PCA Variance Analysis - Left: Scree plot showing explained variance per component; Right: Cumulative explained variance*

## 3. Data Preparation and Feature Engineering

Prior to model training, it is essential to prepare the data appropriately. Machine learning algorithms, particularly Support Vector Machines (SVM) and Artificial Neural Networks (ANN), are sensitive to the scale of input features. Features with larger numerical ranges can dominate the learning process, potentially biasing the model toward those features. Therefore, we apply feature normalization to ensure all features contribute equally to the model's learning process.

### 3.1 Outlier Treatment and Normalization

Our data preparation pipeline consists of two main steps: (1) outlier detection and treatment, and (2) feature standardization. Initial analysis revealed the presence of significant outliers across multiple features, particularly in residual sugar, chlorides, and sulphates. Outliers can significantly affect the mean and standard deviation used in standardization, potentially distorting the transformation. We identify outliers using the Interquartile Range (IQR) method, which is robust to extreme values. Outliers are capped (rather than removed) to preserve the sample size, ensuring we retain all 1,599 samples for model training. Subsequently, we apply standardization (Z-score normalization), which transforms features to have zero mean and unit variance. This standardized dataset will serve as the baseline for all classification models, ensuring fair comparison across different algorithms.

The scree plot and cumulative explained variance plot (see Figure A4 in the appendix) reveal how many components are needed to capture the majority of the variance in the data. The scree plot reveals that the first principal component explains approximately 29% of the total variance, with subsequent components contributing progressively less. The cumulative variance plot indicates that 7 components are required to retain 90% of the variance, while 9 components capture 95% of the total variance. This suggests that dimensionality reduction is feasible, as most information can be preserved with fewer components than the original 11 features.

In [ ]:
from sklearn.preprocessing import StandardScaler
import os

# Create processed data directory
os.makedirs('data/processed', exist_ok=True)

# Separate features and target
X = df.drop('quality', axis=1)
y = df['quality']

# Step 1: Outlier detection using IQR method
print("OUTLIER DETECTION AND TREATMENT")
print("=" * 60)

outliers_info = {}
for feature in X.columns:
    Q1 = X[feature].quantile(0.25)
    Q3 = X[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = ((X[feature] < lower_bound) | (X[feature] > upper_bound)).sum()
    outliers_info[feature] = {
        'count': outliers,
        'percentage': (outliers / len(X)) * 100,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }

# Print outlier summary
outlier_summary = pd.DataFrame({
    'Feature': list(outliers_info.keys()),
    'Outliers': [outliers_info[f]['count'] for f in outliers_info.keys()],
    'Percentage': [f"{outliers_info[f]['percentage']:.2f}%" for f in outliers_info.keys()]
})
print(outlier_summary.to_string(index=False))

# Cap outliers instead of removing them (to preserve sample size)
X_processed = X.copy()
for feature in X.columns:
    Q1 = X[feature].quantile(0.25)
    Q3 = X[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    X_processed[feature] = X_processed[feature].clip(lower=lower_bound, upper=upper_bound)

print(f"\n✓ Outliers capped using IQR method (1.5 * IQR)")
print(f"✓ Original dataset shape: {X.shape}")
print(f"✓ Processed dataset shape: {X_processed.shape}")

In [ ]:
# Step 2: Feature Standardization
print("\nFEATURE STANDARDIZATION")
print("=" * 60)

scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X_processed),
    columns=X_processed.columns,
    index=X_processed.index
)

# Combine scaled features with target
df_normalized = pd.concat([X_scaled, y], axis=1)

# Save normalized dataset
df_normalized.to_csv('data/processed/winequality-red-normalized.csv', index=False)

print("✓ Features standardized using StandardScaler (zero mean, unit variance)")
print(f"✓ Normalized dataset saved to: data/processed/winequality-red-normalized.csv")
print(f"\nNormalized dataset statistics (first 5 features):")
print(X_scaled.describe().iloc[:, :5].round(3))

The normalized dataset (`winequality-red-normalized.csv`) contains all original features standardized to have zero mean and unit variance, with outliers appropriately treated. This dataset will serve as the baseline for all classification models (SVM, ANN, and Random Forest), ensuring that feature scaling does not bias the learning process. The standardization process transforms each feature according to the formula:

$$z = \frac{x - \mu}{\sigma}$$

where $z$ is the standardized value, $x$ is the original value, $\mu$ is the mean, and $\sigma$ is the standard deviation of the feature.

### 3.2 Feature Interactions

The correlation analysis revealed several strong pairwise relationships between features (|r| > 0.5), suggesting that these variables may interact in ways that influence wine quality. While linear models can capture individual feature effects, they may miss how the combination of features together affects the outcome. For example, the effect of fixed acidity on wine quality may depend on the level of citric acid present. To address this, we create multiplicative interaction features for the strongly correlated pairs identified in the correlation matrix. These interactions are created as multiplicative terms between standardized features, allowing us to evaluate whether explicitly modeling feature interactions improves classification performance compared to the baseline normalized dataset.

In [ ]:
# Create interaction features based on strong correlations (|r| > 0.5)
print("FEATURE INTERACTIONS")
print("=" * 60)

# Start with normalized features
X_interactions = X_scaled.copy()

# Define interaction pairs based on strong correlations from correlation matrix
# Positive correlations (r > 0.5)
interaction_pairs = [
    ('fixed acidity', 'citric acid'),
    ('fixed acidity', 'density', ''),
    ('total sulfur dioxide', 'free sulfur dioxide', ''),
    # Negative correlations (r < -0.5)
    ('fixed acidity', 'pH', '_neg'),
    ('citric acid', 'volatile acidity', '_neg'),
    ('citric acid', 'pH', '_neg'),
    ('alcohol', 'density', '_neg')
]

# Create interaction features
interaction_names = []
for feat1, feat2, suffix in interaction_pairs:
    interaction_name = f"{feat1}_{feat2}_interaction{suffix}"
    X_interactions[interaction_name] = X_scaled[feat1] * X_scaled[feat2]
    interaction_names.append(interaction_name)

print(f"✓ Created {len(interaction_pairs)} interaction features:")
for name in interaction_names:
    print(f"  - {name}")

# Combine interaction features with target
df_interactions = pd.concat([X_interactions, y], axis=1)

# Save dataset with interactions
df_interactions.to_csv('data/processed/winequality-red-interactions.csv', index=False)

print(f"\n✓ Dataset with interactions saved to: data/processed/winequality-red-interactions.csv")
print(f"✓ Original features: {X_scaled.shape[1]}")
print(f"✓ Features with interactions: {X_interactions.shape[1]} (added {len(interaction_pairs)} interactions)")
print(f"✓ Dataset shape: {df_interactions.shape}")

The dataset with interactions (`winequality-red-interactions.csv`) contains all original standardized features plus interaction terms for strongly correlated feature pairs. This dataset allows us to evaluate whether capturing explicit feature interactions improves classification performance compared to the baseline normalized dataset.

## 4. Principal Component Analysis (PCA)

Principal Component Analysis (PCA) is a dimensionality reduction technique that transforms the original features into a new set of uncorrelated variables called principal components. These components are ordered by the amount of variance they explain in the data. PCA serves two purposes in our analysis: (1) visualization of high-dimensional data in lower-dimensional space (2D/3D), which is mandatory for this project, and (2) as an alternative dataset for model training, allowing us to compare classification performance with and without dimensionality reduction.

We apply PCA to the normalized dataset to ensure that all features contribute equally to the principal components. The scree plot and cumulative explained variance plot (see Figure A4 in the appendix) reveal how many components are needed to capture the majority of the variance in the data. The analysis shows that 7 components are required to retain 90% of the variance, while 9 components capture 95% of the total variance. This suggests that dimensionality reduction is feasible, as most information can be preserved with fewer components than the original 11 features. The visualization in 2D and 3D space helps us understand the separability of different quality classes in the reduced-dimensional space.

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA to normalized features
print("PRINCIPAL COMPONENT ANALYSIS")
print("=" * 60)

# Fit PCA on normalized data
pca = PCA()
X_pca_full = pca.fit_transform(X_scaled)

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# Determine number of components for 90% and 95% variance retention
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1

print(f"Total features: {X_scaled.shape[1]}")
print(f"\nExplained variance by component:")
for i, (var, cum_var) in enumerate(zip(pca.explained_variance_ratio_, cumulative_variance), 1):
    print(f"  PC{i}: {var:.4f} ({var*100:.2f}%) | Cumulative: {cum_var:.4f} ({cum_var*100:.2f}%)")

print(f"\nComponents needed for 90% variance: {n_components_90}")
print(f"Components needed for 95% variance: {n_components_95}")

# Visualize explained variance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].plot(range(1, len(pca.explained_variance_ratio_) + 1), 
             pca.explained_variance_ratio_, 'bo-', linewidth=2, markersize=8)
axes[0].axhline(y=0.1, color='r', linestyle='--', label='10% threshold')
axes[0].set_xlabel('Principal Component', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Explained Variance Ratio', fontsize=12, fontweight='bold')
axes[0].set_title('Scree Plot', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].legend()

# Cumulative explained variance
axes[1].plot(range(1, len(cumulative_variance) + 1), 
             cumulative_variance, 'go-', linewidth=2, markersize=8)
axes[1].axhline(y=0.90, color='orange', linestyle='--', label='90% threshold')
axes[1].axhline(y=0.95, color='red', linestyle='--', label='95% threshold')
axes[1].set_xlabel('Number of Components', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Cumulative Explained Variance', fontsize=12, fontweight='bold')
axes[1].set_title('Cumulative Explained Variance', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.savefig('figures/pca_variance_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# PCA Visualization (2D and 3D) - MANDATORY
print("\nPCA VISUALIZATION (2D and 3D)")
print("=" * 60)

# Use first 2 and 3 components for visualization
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_scaled)

# 2D Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 2D scatter plot
scatter = axes[0].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y, cmap='viridis', 
                          alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel(f'First Principal Component ({pca_2d.explained_variance_ratio_[0]*100:.2f}% variance)', 
                   fontsize=12, fontweight='bold')
axes[0].set_ylabel(f'Second Principal Component ({pca_2d.explained_variance_ratio_[1]*100:.2f}% variance)', 
                   fontsize=12, fontweight='bold')
axes[0].set_title('PCA 2D Visualization', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
cbar = plt.colorbar(scatter, ax=axes[0])
cbar.set_label('Quality Score', fontsize=10, fontweight='bold')

# 3D Visualization
ax_3d = fig.add_subplot(122, projection='3d')
scatter_3d = ax_3d.scatter(X_pca_3d[:, 0], X_pca_3d[:, 1], X_pca_3d[:, 2], 
                          c=y, cmap='viridis', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
ax_3d.set_xlabel(f'PC1 ({pca_3d.explained_variance_ratio_[0]*100:.2f}%)', fontsize=10, fontweight='bold')
ax_3d.set_ylabel(f'PC2 ({pca_3d.explained_variance_ratio_[1]*100:.2f}%)', fontsize=10, fontweight='bold')
ax_3d.set_zlabel(f'PC3 ({pca_3d.explained_variance_ratio_[2]*100:.2f}%)', fontsize=10, fontweight='bold')
ax_3d.set_title('PCA 3D Visualization', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('figures/pca_visualization_2d_3d.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ 2D visualization: PC1 ({pca_2d.explained_variance_ratio_[0]*100:.2f}%) + PC2 ({pca_2d.explained_variance_ratio_[1]*100:.2f}%) = {sum(pca_2d.explained_variance_ratio_)*100:.2f}% total variance")
print(f"✓ 3D visualization: PC1 ({pca_3d.explained_variance_ratio_[0]*100:.2f}%) + PC2 ({pca_3d.explained_variance_ratio_[1]*100:.2f}%) + PC3 ({pca_3d.explained_variance_ratio_[2]*100:.2f}%) = {sum(pca_3d.explained_variance_ratio_)*100:.2f}% total variance")

In [ ]:
# Create PCA dataset for model training (using components that retain 95% variance)
print("\nCREATING PCA DATASET FOR MODEL TRAINING")
print("=" * 60)

pca_model = PCA(n_components=n_components_95)
X_pca_model = pca_model.fit_transform(X_scaled)

# Create DataFrame with principal components
pca_columns = [f'PC{i+1}' for i in range(n_components_95)]
df_pca = pd.DataFrame(X_pca_model, columns=pca_columns, index=X_scaled.index)
df_pca = pd.concat([df_pca, y], axis=1)

# Save PCA dataset
df_pca.to_csv('data/processed/winequality-red-pca.csv', index=False)

print(f"✓ PCA dataset created with {n_components_95} components (retaining {cumulative_variance[n_components_95-1]*100:.2f}% variance)")
print(f"✓ PCA dataset saved to: data/processed/winequality-red-pca.csv")
print(f"\nPCA dataset shape: {df_pca.shape}")
print(f"Original features: {X_scaled.shape[1]} → PCA components: {n_components_95} (reduction of {X_scaled.shape[1] - n_components_95} features)")

The PCA visualization reveals the separability of different quality classes in the reduced-dimensional space. While perfect separation is not expected in 2D or 3D projections (as they capture only a portion of the total variance), the visualization provides insights into the underlying structure of the data. The PCA-transformed dataset (`winequality-red-pca.csv`) will be used as an alternative input for classification models, allowing us to compare performance with the normalized dataset and assess whether dimensionality reduction improves or hinders classification accuracy.

### A3. Feature Correlations with Quality Score

A detailed analysis focusing specifically on feature correlations with the quality score is presented below, providing a focused perspective on feature-target relationships.

In [ ]:
# Correlation with target variable (quality) - for appendix
quality_corr = df.corr()['quality'].sort_values(ascending=False)
quality_corr = quality_corr.drop('quality')

plt.figure(figsize=(10, 6))
colors = ['red' if x < 0 else 'green' for x in quality_corr.values]
bars = plt.barh(quality_corr.index, quality_corr.values, color=colors, alpha=0.7)
plt.xlabel('Correlation with Quality', fontsize=12, fontweight='bold')
plt.title('Feature Correlation with Quality Score', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, (idx, val) in enumerate(quality_corr.items()):
    plt.text(val + 0.01 if val > 0 else val - 0.01, i, f'{val:.3f}', 
             va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('figures/quality_correlation_bar.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nTop features correlated with quality:")
print(quality_corr.abs().sort_values(ascending=False))